In [1]:
!pip install selenium
!pip install beautifulsoup4

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup

# Initialize the Chrome WebDriver
driver = webdriver.Chrome()
driver.get("https://property.spatialest.com/co/elpaso/#/")

WebDriverWait(driver, 10).until(
    EC.visibility_of_element_located((By.ID, "primary_search"))
)

# Wait for the search box (address input field) to be visible
search_box = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.ID, 'primary_search'))
)

# Send the address to the search box (e.g., "1234 Main St")
search_box.send_keys("16275 Eaglenest Dr")

# Simulate hitting the Enter key to trigger the search
search_box.send_keys(Keys.RETURN)

# Wait explicitly for the data list section to be visible
WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "data-list-section"))
)

# Grab the page source after the search results load
page_source = driver.page_source

# Use BeautifulSoup to parse the page
soup = BeautifulSoup(page_source, 'html.parser')
# soup.prettify()


### Overview


In [3]:
# Find the 'data-list-section' that contains the relevant data
data_list_section = soup.find('div', class_='data-list-section')

# Find all the list items (li) within this section
data_rows = data_list_section.find_all('li', class_='clearfix data-list-row')

property_details = {}

# Iterate over each data row to extract the title and value
for row in data_rows:
    title = row.find('span', class_='title')
    value = row.find('span', class_='value')
    
    # If the value is a dropdown (select), extract the selected option
    if not value:
        value = row.find('select')
        if value:
            value = value.find('option').text.strip()
    else:
        value = value.text.strip()
    
    if title and value:
        title_text = title.text.strip()
        property_details[title_text] = value

# Print out the extracted data
for key, value in property_details.items():
    print(f"{key} {value}")


Owner: MARTIN ROBERT GUY
Mailing Address: 16275 EAGLENEST DR COLORADO SPRINGS CO, 80930-9415
Location: 16275 EAGLENEST DR
Tax Status: Taxable
Zoning: RR-5
Plat No: -
Legal Description: TRACT IN S2 SEC 11-14-64 AS FOLS, COM AT E4 OF SD SEC, TH N 89<51' W ON E-W C/L 2370.07 FT, S 0<31' E 1313.60 FT FOR POB, CONT SLY ON LAST COURSE 660.0 FT, S 89<57' W 640.0 FT, N 56<33' W 712.63 FT, N 0<31' W 581.48 FT, S 74<33' E 1080.0 FT, TH ON A CUR TO L HAVING A RAD OF 720.0 FT + C/A OF 15<30' AN ARC DIST OF 194.78 FT, TH N 89<57' E 0.48 FT TO POB TOG WITH EASEMENT FOR INGRESS + EGRESS AS DES IN BK 2629-779


### Market & Assessment Details

In [4]:
# Find the "Market & Assessment Details" section
assessment_section = soup.find('div', class_='assessment')

# Find the data list containing market and assessed values
data_list = assessment_section.find('ul', class_='data-list')

# Extract the rows for the market and assessed values
rows = data_list.find_all('li', class_='clearfix data-list-row')

market_value = {}
assessed_value = {}

for row in rows:
    title = row.find('span', class_='title')
    value = row.find('span', class_='value')
    
    if title and value:
        title_text = title.text.strip()
        value_text = value.text.strip()

        # Checking if the title matches categories and storing the respective values
        if title_text in ["Land", "Improvement", "Total"]:
            market_value[title_text] = value_text
        if title_text in ["Land", "Improvement", "Total"]:
            assessed_value[title_text] = value_text

# Print the results for Market and Assessed Values
print("Market Value:")
for key, val in market_value.items():
    print(f"{key}: {val}")

print("\nAssessed Value:")
for key, val in assessed_value.items():
    print(f"{key}: {val}")

Market Value:
Land: $171,700
Improvement: $612,655
Total: $784,355

Assessed Value:
Land: $171,700
Improvement: $612,655
Total: $784,355


### Land Details

In [5]:
# Find the table containing the land details
table = soup.find('table', class_='table-striped')

# Extract the headers (columns) from the table
headers = [th.text.strip() for th in table.find_all('th')]

# Extract the rows of the table
rows = table.find_all('tr')

# Initialize an empty list to store dictionaries for each row
data = []

# Loop through each row (skipping the header row)
for row in rows[1:]:
    columns = row.find_all('td')
    
    if columns:
        row_data = {}
        for i, col in enumerate(columns):
            # Assign the header as the key and the column value as the value
            row_data[headers[i]] = col.text.strip()
        data.append(row_data)

# Now print the scraped land info in the format you requested
for land_info in data:
    for key, value in land_info.items():
        print(f"{key}: {value}")
    print()

Sequence Number: 1
Land Use: SINGLE FAMILY RESIDENTIAL
Assessment Rate: 6.700
Area: 20.09 Acres
Market Value: $166,700

Sequence Number: 2
Land Use: WELL AND SEPTIC CONVERSION VALUE
Assessment Rate: 6.700
Area: 0 SQFT
Market Value: $5,000



### Building Details

In [11]:
sections = soup.find_all('div', class_='panel panel-default')

# Loop through each section and extract the details
for section in sections:
    # Extract title for identification (either BI LEVEL 2 STORY or RESIDENTIAL OUTBUILDINGS)
    section_title = section.find('h4', class_='panel-title').get_text(strip=True)

    # Extract building details for each section
    market_value = section.find('div', class_='building-value').find_all('span')[1].get_text(strip=True)
    data_list = section.find('ul', class_='data-list')
    building_details = {}

    # Loop through each row and extract the title and value for each <p> tag
    for item in data_list.find_all('li', class_='data-list-row'):
        data_items = item.find_all('p', class_='data-list-item')
        for data_item in data_items:
            title_span = data_item.find('span', class_='title')
            value_span = data_item.find('span', class_='value')

            if title_span:
                title = title_span.get_text(strip=True)
                value = value_span.get_text(strip=True) if value_span else "-"
                building_details[title] = value
    
    # Add the market value to the building details dictionary
    building_details['Market Value'] = market_value

    # Print out the details for each section
    print(f"\n{section_title} Details:")
    for key, value in building_details.items():
        print(f"{key}: {value}")


BI LEVEL 2 STORY (1) Details:
Assessment Rate: 6.700
Above Grade Area: 2,765
Bldg #: 1
First Floor Area: 1,965
Style Description: BI LEVEL 2 STORY
Above First Floor Area: 800
Property Description: SINGLE FAMILY RESIDENTIAL
Lower Level Living Area: 0
Year Built: 1978
Total Basement Area: 876
Dwelling Units: 1
Finished Basement Area: 876
Number of Rooms: 7
Garage Description: Attached
Number of Bedrooms: 3
Garage Area: 744
Number of Baths: 1.75
Carport Area: -
Market Value: $612,655

RESIDENTIAL OUTBUILDINGS (2) Details:
Assessment Rate: 6.700
Sprinkler: N
Bldg #: 2
Elevator: -
Use: RESIDENTIAL OUTBUILDINGS
Occup 1: 430
Year Built: 2020
Occup 2: -
Area: 576
HVA 1: Electric Baseboard
Class: D
HVA 2: -
Quality: Average
Wall Height: 8
Stories: -
Land Size: -
Perimeter: -
Neigh #: 97
# Units: -
Market Value: $0


### Sales History

In [7]:
# Find the sales table
sales_history = soup.find('div', id='sales')
sales_table = sales_history.find_all('tr')

# Initialize a list to store the sales data
sales_data = []

# Loop through each row in the sales table
for row in sales_table:
    sale_info = {}
    
    # Extract the sale date, price, type, and reception from the main row
    columns = row.find_all('td')
    if len(columns) > 1:
        sale_date = columns[1].get_text(strip=True)
        sale_price = columns[2].get_text(strip=True)
        sale_type = columns[3].get_text(strip=True)
        reception = columns[4].get_text(strip=True)
        
        # Store the extracted information in the dictionary
        sale_info['Sale Date'] = sale_date
        sale_info['Sale Price'] = sale_price
        sale_info['Sale Type'] = sale_type
        sale_info['Reception'] = reception
        
        # Check for additional sale details if available (expand row button +)
        expand_row = columns[0].find('button')
        if expand_row:
            # Simulate a click to reveal additional information
            expand_button = driver.find_element(By.XPATH, f"//button[text()='+']")
            expand_button.click()
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CLASS_NAME, "table-row-subdata-content"))
            )
            
            # Refresh the page source after expanding
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            
            # Re-locate the expanded data in the DOM
            expanded_row = soup.find_all('tr', class_='hide table-row-subdata')

            # Extract subdata for each expanded row
            if expanded_row:
                subdata = expanded_row[0].find('ul', class_='data-list')
                if subdata:
                    for item in subdata.find_all('li', class_='data-list-row'):
                        # Extract the title and value from each <p> tag
                        data_items = item.find_all('p', class_='data-list-item')
                        for data_item in data_items:
                            title_span = data_item.find('span', class_='title')
                            value_span = data_item.find('span', class_='value')

                            if title_span and value_span:
                                title = title_span.get_text(strip=True)
                                value = value_span.get_text(strip=True)
                                sale_info[title] = value
                    
                    # Now, handle the Grantee field (dropdown)
                    grantee_select = subdata.find('select', class_='value')
                    if grantee_select:
                        # Get the first option in the dropdown, which is visible
                        selected_grantee = grantee_select.find('option')
                        if selected_grantee:
                            sale_info['Grantee'] = selected_grantee.get_text(strip=True)
        
        # Append the sale data to the list
        sales_data.append(sale_info)

# Print the sales data in a cleaner format
for sale in sales_data:
    for key, value in sale.items():
        print(f"{key}: {value}")
    print()

# Close the WebDriver
# driver.quit()

Sale Date: 12/01/2016
Sale Price: $177,000
Sale Type: Arms-Length Sale
Reception: 216139263
Schedule No: 4411000017
Book: -
Page: -
Balloon: No
PP/Good Will: $0
Related Parties: N
Trade/Exch: $0
Condition: Average
Term: Month(s): 360
Financing: New 3.25% Fixed
Amt. Financed: $182,828
Down Pmt: $0
Doc Type: WARRANTY DEED
Grantor: WEBB RONALD D
Grantee: MARTIN ROBERT GUY

Sale Date: 01/03/2001
Sale Price: $0
Sale Type: -
Reception: 216139263
Schedule No: 4411000017
Book: -
Page: -
Balloon: No
PP/Good Will: $0
Related Parties: N
Trade/Exch: $0
Condition: Average
Term: Month(s): 360
Financing: New 3.25% Fixed
Amt. Financed: $182,828
Down Pmt: $0
Doc Type: WARRANTY DEED
Grantor: WEBB RONALD D
Grantee: MARTIN ROBERT GUY

Sale Date: 05/01/1979
Sale Price: $0
Sale Type: -
Reception: -



### Tax Entity and Levy Information

In [10]:
# Find the Tax Entity and Levy Information section
tax_levy_section = soup.find('div', {'id': 'taxandlevytab'})

# Extract the tax area code, levy year, and mill levy from the paragraph
tax_info = tax_levy_section.find_all('p')[1].get_text(strip=True)
print(f"Tax Information: \n   {tax_info} \n")

# Extract all rows from the Taxing Entity table
table_rows = tax_levy_section.find_all('tr')

# Initialize a list to store sales data
sales_data = []

# Extract and store the table data for each row
for row in table_rows:
    cols = row.find_all('td')
    if len(cols) > 0:  # Skip empty rows
        sale = {
            "Taxing Entity": cols[0].get_text(strip=True),
            "Levy": cols[1].get_text(strip=True),
            "Contact Name/Organization": cols[2].get_text(strip=True),
            "Contact Phone": cols[3].get_text(strip=True)
        }
        sales_data.append(sale)

# Print the sales data in the requested format
for sale in sales_data:
    for key, value in sale.items():
        print(f"{key}: {value}")
    print()

Tax Information: 
   Tax Area Code:KB4Levy Year:2024Mill Levy:56.668 

Taxing Entity: EL PASO COUNTY
Levy: 6.985
Contact Name/Organization: FINANCIAL SERVICES
Contact Phone: (719)520-6400

Taxing Entity: EPC ROAD & BRIDGE (UNSHARED)
Levy: 0.330
Contact Name/Organization: -
Contact Phone: (719)520-6498

Taxing Entity: ELLICOTT SCHOOL DISTRICT #22
Levy: 29.901
Contact Name/Organization: AMANDA KOBILAN
Contact Phone: (719)683-2700

Taxing Entity: PIKES PEAK LIBRARY DISTRICT
Levy: 3.140
Contact Name/Organization: RANDALL A GREEN
Contact Phone: (719)531-6333

Taxing Entity: ELLICOTT FIRE PROTECTION DISTRICT
Levy: 15.230
Contact Name/Organization: MICHAEL HENLEY
Contact Phone: (719)683-7211

Taxing Entity: UPPER BLK SQUIRREL CRK GROUND WATER DISTRICT
Levy: 1.082
Contact Name/Organization: TRACY DORAN
Contact Phone: (719)510-0780

Taxing Entity: ELLICOTT METRO DISTRICT
Levy: 0.000
Contact Name/Organization: GEORGIA MCREA
Contact Phone: (719)683-4190

Taxing Entity: EL PASO COUNTY CONSERVATION

### Map Sheet

In [9]:
# Find the MapSheet div
map_sheet_div = soup.find('div', {'id': 'MapSheet'})

# Find the <a> tag within the MapSheet div
map_link = map_sheet_div.find('a')

# Extract the href (link) and text from the <a> tag
if map_link:
    map_url = map_link.get('href')
    map_text = map_link.get_text(strip=True)
    print(f"Map URL: {map_url}")
    # print(f"Map Text: {map_text}")
else:
    print("Map link not found.")

# Close the driver
driver.quit()

Map URL: https://image-cdn.spatialest.com/file/view/co-elpaso-images/ASRMap/44110.tif?v=210622
